In [1]:
# %% Import Libraries
#This is a test
import os
import pickle
import scipy.io
import numpy as np
import pandas as pd
import torch as tc
import matplotlib.pyplot as plt

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split

import matplotlib.patches as mpatches
from tqdm import tqdm
from bptt.models import Model
import model_anafunctions as func

plt.rcParams['font.size'] = 20

c:\Users\cestarellas\anaconda3\envs\PLAna\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#%% FUNCTIONS
def Hyper_mod(mpath,data_path):
    file=open(os.path.join(mpath,'hypers.pkl').replace("\\","/"),'rb')
    hyper=pickle.load(file)
    file.close()
    hyper['data_path']=os.path.join(data_path,'Training_data.npy').replace('\\','/')
    hyper['inputs_path']=os.path.join(data_path,'Training_inputs.npy').replace('\\','/')
    hyper['device_id'] = 0
    full_name = open(os.path.join(mpath,'hypers.pkl').replace("\\","/"),"wb")                      # Name for training data
    pickle.dump(hyper,full_name)            # Save train data
    #close save instance 
    full_name.close()

In [3]:
#%% Data Organization
# General Directories
path_models = 'D:\\_work_cestarellas\\Analysis\\Pack_Daniel_project\\Preprocess_model\\' # For models and Training and Test data
path_behaviours = 'D:\\_work_cestarellas\\Analysis\\PLRNN\\Session_Selected\\OFC'        # For behavioural sessions

#Model
mod_sessions = os.listdir(path_models)
extra_models = 'results\\DataTrainingH768_lm1_1e-05_lm2_128_lm3_00_seql_400\\001'
#Neuronal Activity
extra_activity = 'neuralactivity\\datasets\\'
#Behaviour
rec_sessions = os.listdir(path_behaviours)

#################### WARNING
# Be careful!!! Check if the order of mod_sessions and rec_sessions is the same
#################### WARNING

In [4]:
mod_sessions

['bptt_CE17', 'bptt_DM01_5', 'bptt_DM01_7', 'bptt_JG15_24', 'bptt_JG15_25']

In [5]:
rec_sessions

['CE17_L6',
 'DM01_5_220522',
 'DM01_7_220524',
 'JG15_190724_clustered',
 'JG15_190725_clustered']

In [6]:
def Generation_Matrixes(data_path,mpath,num_epoch,event):
    # Loading Train and Test data    
    train_n,train_i = func.load_data(data_path,'Training')
    test_n,test_i = func.load_data(data_path,'Test')

    # Loading Model
    num_epochs = num_epoch
    m = Model()
    m.init_from_model_path(mpath, epoch=num_epochs)
    m.eval()

    # Generation Training Data
    ModelS=[]
    for w_index in tqdm(range(len(train_n))):
        data_trial=tc.from_numpy(train_n[w_index]).float()          # tensor of neuronal data for initial trial data
        input_trial = tc.from_numpy(train_i[w_index]).float()
        length_sim = input_trial.shape[0]
        X, _ = m.generate_free_trajectory(data_trial,input_trial,length_sim,w_index)
        ModelS.append(X[:,:])

    # Concatenating signals
    Nseries,_=func.concatenate_list(train_n,0)          # recorded activity
    Iseries,_=func.concatenate_list(train_i,0)          # external input
    Mseries,_=func.concatenate_list(ModelS,0)           # simulated activity
    
    if event == 1:
        # Temporal sectors: Period of time selected to compute the mean firing rate
        Time_ini = np.where(np.diff(Iseries[:,0])==1)[0]-49                        # Initial point:beginning wheel stop
        Time_end = np.where(np.diff(Iseries[:,0])==-1)[0]                          # Final point: ending execution time. Just before reward
    elif event == 2:
        # Temporal sectors: Period of time selected to compute the mean firing rate
        Time_ini = np.where(np.diff(Iseries[:,0])==-1)[0]                       # Initial point: beginning reward
        RT_end = np.where(np.diff(Iseries[:,0])==1)[0]-49
        Time_end = np.append(RT_end[1:],Iseries.shape[0])                       # Final point: ending Inter-Trial Interval (beggining following trial)
    elif event == 3:
        # Temporal sectors: Period of time selected to compute the mean firing rate
        Time_ini = np.where(np.diff(Iseries[:,0])==1)[0]-49                 # Initial point: Beginning trial
        Time_end = np.append(Time_ini[1:],Iseries.shape[0])                   # Final point: Ending trial

    
    # Data Set Mean Activity Zscore Neurons
    num_trials = len(Time_end)
    num_neurons = Nseries.shape[1]
    X_data = np.zeros((num_trials,num_neurons))
    X_model = np.zeros((num_trials,num_neurons))
    for it in range(num_trials):
        X_data[it,:]=np.mean(Nseries[Time_ini[it]:Time_end[it],:],0)
        X_model[it,:]=np.mean(Mseries[Time_ini[it]:Time_end[it],:],0)
    
    return X_data,X_model

In [11]:
num_sessions = len(mod_sessions)
last_epoch = 150000
Choice_rec = [] ; Choice_gen =[]
Reward_rec = [] ; Reward_gen =[]
Block_rec = [] ; Block_gen =[]


for isess in range(num_sessions):
    Neural_path = os.path.join(path_models,mod_sessions[isess],extra_activity)
    Model_path = os.path.join(path_models,mod_sessions[isess],extra_models).replace('\\','/')
    Behavioural_path = os.path.join(path_behaviours,rec_sessions[isess]).replace('\\','/')
    Hyper_mod(Model_path,Neural_path)
    
    # Response mean value
    event_flag=1
    Data_resp,Simul_resp = Generation_Matrixes(Neural_path,Model_path,last_epoch,event_flag)
    scipy.io.savemat('Resp_'+mod_sessions[isess]+'.mat',{'Rec':Data_resp,'Gen':Simul_resp})
    # Reward mean value
    event_flag=2
    Data_rew,Simul_rew = Generation_Matrixes(Neural_path,Model_path,last_epoch,event_flag)
    scipy.io.savemat('Rew_'+mod_sessions[isess]+'.mat',{'Rec':Data_rew,'Gen':Simul_rew})
    # Total trial mean value
    event_flag=3
    Data_tot,Simul_tot = Generation_Matrixes(Neural_path,Model_path,last_epoch,event_flag)
    scipy.io.savemat('Tot_'+mod_sessions[isess]+'.mat',{'Rec':Data_tot,'Gen':Simul_tot})
    

Chosen model is clipped shPLRNN, the bases Parameter has no effect here!


100%|██████████████████████████████████████████████████████████████████████████████████| 55/55 [00:05<00:00,  9.38it/s]


Chosen model is clipped shPLRNN, the bases Parameter has no effect here!


100%|██████████████████████████████████████████████████████████████████████████████████| 55/55 [00:05<00:00,  9.21it/s]


Chosen model is clipped shPLRNN, the bases Parameter has no effect here!


...

100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [00:03<00:00,  9.65it/s]


Chosen model is clipped shPLRNN, the bases Parameter has no effect here!


100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [00:03<00:00,  9.66it/s]


Chosen model is clipped shPLRNN, the bases Parameter has no effect here!


100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [00:03<00:00,  9.64it/s]


In [ ]:
pwd